In [0]:
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
# Set Azure blob configuration
spark.conf.set(
  "somecontainer@somestorage",
  "somekey")

# Read parquet and create dataframe
trade_common = spark.read.format('parquet').load("wasbs://somecontainer@somestorage.blob.core.windows.net/output_dir/partition=T")

In [0]:
# Retrieve relevant fields
trade = trade_common.select("trade_dt", "rec_type", "symbol", "exchange", "event_tm", "event_seq_nb", "arrival_tm", "trade_pr")

In [0]:
trade.show(10)

+----------+--------+------+--------+--------------------+------------+-------------------+---------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+
2020-08-05| T| SYMB| NASDAQ|2020-08-05 16:29:...| 60|2020-08-05 09:30:00|34.867146|
2020-08-05| T| SYMB| NASDAQ|2020-08-05 17:42:...| 70|2020-08-05 09:30:00|36.291695|
2020-08-05| T| SYMB| NASDAQ|2020-08-05 18:56:...| 80|2020-08-05 09:30:00|35.614357|
2020-08-05| T| SYMB| NASDAQ|2020-08-05 20:04:...| 90|2020-08-05 09:30:00|36.751675|
2020-08-05| T| SYMB| NASDAQ|2020-08-05 21:03:...| 100|2020-08-05 09:30:00|35.537262|
2020-08-05| T| SYMC| NASDAQ|2020-08-05 10:47:...| 10|2020-08-05 09:30:00|159.57423|
2020-08-05| T| SYMC| NASDAQ|2020-08-05 12:00:...| 20|2020-08-05 09:30:00|159.82744|
2020-08-05| T| SYMC| NASDAQ|2020-08-05 13:13:...| 30|2020-08-05 09:30:00|157.85478|
2020-08-05| T| SYMC| NASDAQ|2020-08-05 14:36:...| 40|2020-08-05 09:30:00|158.63351|
2020-08-05| T| SYMC| NASDAQ|2020-08-05 15:49:...| 50|2020-08-05 09:30:00|157.06882|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+
only showing top 10 rows

In [0]:
def apply_latest(df):
  
    if df.first()["rec_type"] == "T":
    
        # Group records based on latest trade date
        df_grouped = df.groupBy("trade_dt", "rec_type", "symbol", "arrival_tm", "event_seq_nb").agg(max("event_tm").alias("latest_trade"))
    
        # Join with original dataframe to retrieve exchange and trade_pr for latest trade date
        df_joined = df_grouped.join(df.select("event_tm", "exchange", "trade_pr"), df.event_tm == df_grouped.latest_trade, "inner")

        # Retrieve relevant fields
        df_final = df_joined.select("trade_dt", "rec_type", col("symbol").alias("stock_symbol"), col("exchange").alias("stock_exchange"), "latest_trade", "event_seq_nb", "arrival_tm", "trade_pr").orderBy("trade_dt", "symbol", "event_seq_nb")

        return df_final
  
    elif df.first()["rec_type"] == "Q":
  
        # Group records based on latest trade date
        df_grouped = df.groupBy("trade_dt", "rec_type", "symbol", "arrival_tm", "event_seq_nb").agg(max("event_tm").alias("latest_quote"))
    
        # Join with original dataframe to retrieve exchange, bid_pr, bid_size, ask_pr and ask_size
        df_joined = df_grouped.join(df.select("event_tm", "exchange", "bid_pr", "bid_size", "ask_pr", "ask_size"), df.event_tm == df_grouped.latest_quote, "inner")
    
        # Retrieve relevant fields
        df_final = df_joined.select("trade_dt", "rec_type", col("symbol").alias("stock_symbol"), col("exchange").alias("stock_exchange"), "latest_quote", "event_seq_nb", "arrival_tm", "bid_pr", "bid_size", "ask_pr", "ask_size").orderBy("trade_dt", "symbol", "event_seq_nb")
    
        return df_final

In [0]:
trade_corrected = apply_latest(trade)

In [0]:
trade_corrected.show(10)

+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+
 trade_dt|rec_type|stock_symbol|stock_exchange| latest_trade|event_seq_nb| arrival_tm| trade_pr|
+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+
2020-08-05| T| SYMA| NASDAQ|2020-08-05 10:38:...| 10|2020-08-05 09:30:00| 77.7757|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 11:58:...| 20|2020-08-05 09:30:00|75.715225|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 13:09:...| 30|2020-08-05 09:30:00| 75.87926|
2020-08-05| T| SYMA| NYSE|2020-08-05 14:24:...| 40|2020-08-05 09:30:00|78.430954|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 15:33:...| 50|2020-08-05 09:30:00| 75.72602|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 16:46:...| 60|2020-08-05 09:30:00|77.479485|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 17:58:...| 70|2020-08-05 09:30:00| 74.91233|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 19:13:...| 80|2020-08-05 09:30:00| 77.69288|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 20:26:...| 90|2020-08-05 09:30:00| 76.15561|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 21:40:...| 100|2020-08-05 09:30:00| 77.24676|
+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+
only showing top 10 rows

In [0]:
# Separate dataframes by trade date
trade_corrected_05 = trade_corrected.where(trade_corrected.trade_dt == "2020-08-05")
trade_corrected_06 = trade_corrected.where(trade_corrected.trade_dt == "2020-08-06")

In [0]:
trade_corrected_05.show()

+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+
 trade_dt|rec_type|stock_symbol|stock_exchange| latest_trade|event_seq_nb| arrival_tm| trade_pr|
+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+
2020-08-05| T| SYMA| NASDAQ|2020-08-05 10:38:...| 10|2020-08-05 09:30:00| 77.7757|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 11:58:...| 20|2020-08-05 09:30:00|75.715225|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 13:09:...| 30|2020-08-05 09:30:00| 75.87926|
2020-08-05| T| SYMA| NYSE|2020-08-05 14:24:...| 40|2020-08-05 09:30:00|78.430954|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 15:33:...| 50|2020-08-05 09:30:00| 75.72602|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 16:46:...| 60|2020-08-05 09:30:00|77.479485|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 17:58:...| 70|2020-08-05 09:30:00| 74.91233|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 19:13:...| 80|2020-08-05 09:30:00| 77.69288|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 20:26:...| 90|2020-08-05 09:30:00| 76.15561|
2020-08-05| T| SYMA| NASDAQ|2020-08-05 21:40:...| 100|2020-08-05 09:30:00| 77.24676|
2020-08-05| T| SYMB| NYSE|2020-08-05 10:43:...| 10|2020-08-05 09:30:00|34.978676|
2020-08-05| T| SYMB| NYSE|2020-08-05 12:02:...| 20|2020-08-05 09:30:00| 33.17828|
2020-08-05| T| SYMB| NYSE|2020-08-05 13:10:...| 30|2020-08-05 09:30:00|34.183914|
2020-08-05| T| SYMB| NYSE|2020-08-05 14:26:...| 40|2020-08-05 09:30:00|36.511795|
2020-08-05| T| SYMB| NYSE|2020-08-05 15:31:...| 50|2020-08-05 09:30:00| 36.07549|
2020-08-05| T| SYMB| NYSE|2020-08-05 16:48:...| 60|2020-08-05 09:30:00|35.762054|
2020-08-05| T| SYMB| NYSE|2020-08-05 18:02:...| 70|2020-08-05 09:30:00|32.681087|
2020-08-05| T| SYMB| NYSE|2020-08-05 19:08:...| 80|2020-08-05 09:30:00| 34.27408|
2020-08-05| T| SYMB| NYSE|2020-08-05 20:17:...| 90|2020-08-05 09:30:00| 34.2493|
2020-08-05| T| SYMB| NYSE|2020-08-05 21:27:...| 100|2020-08-05 09:30:00|33.956287|
+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+
only showing top 20 rows

In [0]:
# Write to Azure blob in parquet format
trade_corrected_05.write.parquet("wasbs://somecontainer@somestorage.blob.core.windows.net/trade/trade_dt={}".format('2020-08-05'))
trade_corrected_06.write.parquet("wasbs://somecontainer@somestorage.blob.core.windows.net/trade/trade_dt={}".format('2020-08-06'))

In [0]:
# Set Azure blob configuration
spark.conf.set(
  "somecontainer@somestorage",
  "somekey")

# Read parquet and create dataframe
quote_common = spark.read.format('parquet').load("wasbs://somecontainer@somestorage.blob.core.windows.net/output_dir/partition=Q")

In [0]:
# Retrieve relevant fields
quote = quote_common.select("trade_dt", "rec_type", "symbol", "exchange", "event_tm", "event_seq_nb", "arrival_tm", "bid_pr", "bid_size", "ask_pr", "ask_size")

In [0]:
quote.show(10)

+----------+--------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm| bid_pr|bid_size| ask_pr|ask_size|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:36:...| 1|2020-08-05 09:30:00| 76.10017| 100| 77.9648| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:42:...| 2|2020-08-05 09:30:00| 75.44373| 100| 75.94453| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:48:...| 3|2020-08-05 09:30:00|78.847984| 100| 80.69115| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:53:...| 4|2020-08-05 09:30:00| 74.98337| 100| 76.16257| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:00:...| 5|2020-08-05 09:30:00| 76.71408| 100|77.855125| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:05:...| 6|2020-08-05 09:30:00| 76.78901| 100| 77.79983| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:13:...| 7|2020-08-05 09:30:00|76.370995| 100| 76.94632| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:21:...| 8|2020-08-05 09:30:00| 77.32601| 100| 77.65018| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:30:...| 9|2020-08-05 09:30:00| 77.11395| 100| 77.52986| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:45:...| 11|2020-08-05 09:30:00| 77.69893| 100| 79.18605| 100|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+
only showing top 10 rows

In [0]:
quote_corrected = apply_latest(quote)

In [0]:
quote_corrected.show(10)

+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+--------+---------+--------+
 trade_dt|rec_type|stock_symbol|stock_exchange| latest_quote|event_seq_nb| arrival_tm| bid_pr|bid_size| ask_pr|ask_size|
+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+--------+---------+--------+
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:36:...| 1|2020-08-05 09:30:00| 76.10017| 100| 77.9648| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 09:42:...| 2|2020-08-05 09:30:00| 75.44373| 100| 75.94453| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:50:...| 3|2020-08-05 09:30:00| 77.15973| 100| 77.33205| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:57:...| 4|2020-08-05 09:30:00|79.299774| 100| 80.08399| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:06:...| 5|2020-08-05 09:30:00|77.863495| 100| 78.30821| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:11:...| 6|2020-08-05 09:30:00| 78.74799| 100|80.329056| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:17:...| 7|2020-08-05 09:30:00| 77.16837| 100| 77.97598| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:23:...| 8|2020-08-05 09:30:00| 78.06551| 100| 78.80516| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:31:...| 9|2020-08-05 09:30:00| 76.69653| 100| 78.56943| 100|
2020-08-05| Q| SYMA| NASDAQ|2020-08-05 10:45:...| 11|2020-08-05 09:30:00| 77.69893| 100| 79.18605| 100|
+----------+--------+------------+--------------+--------------------+------------+-------------------+---------+--------+---------+--------+
only showing top 10 rows

In [0]:
# Separate dataframes by trade date
quote_corrected_05 = quote_corrected.where(quote_corrected.trade_dt == "2020-08-05")
quote_corrected_06 = quote_corrected.where(quote_corrected.trade_dt == "2020-08-06")

In [0]:
# Write to Azure blob in parquet format
quote_corrected_05.write.parquet("wasbs://somecontainer@somestorage.blob.core.windows.net/quote/trade_dt={}".format('2020-08-05'))
quote_corrected_06.write.parquet("wasbs://somecontainer@somestorage.blob.core.windows.net/quote/trade_dt={}".format('2020-08-06'))